<a href="https://colab.research.google.com/github/sirthirrygolooo/Notebooks/blob/main/noSQL/hehe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installer les bibliothèques

In [2]:
%pip install semantic-link
%pip install pyspark
%pip install azure-keyvault-secrets==4.7.0
%pip install azure-identity
%pip install azure-mgmt-resource
%pip install azure-storage-blob
%pip install azure-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.5/297.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.6/348.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.2/113.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/4

In [3]:
import pandas as pd
import sempy.fabric as fabric
from sempy.fabric import FabricDataFrame
from sempy.dependencies import plot_dependency_metadata
import re

log_telemetry: sempy.relationships
log_telemetry: sempy.dependencies
log_telemetry: sempy.fabric
log_telemetry: sempy


### Préparation des données

In [4]:
data_emp = """
PROJ_NUM PROJ_NAME EMP_NUM EMP_NAME CHG_HOUR JOB_CLASS HOURS
15 Evergreen 103 June E. Arbough 84.50 Elect. Engineer 23.8
15 Evergreen 101 John G. News 105.00 Database Designer 19.4
15 Evergreen 105 Alice K. Johnson 105.00 Database Designer 35.7
15 Evergreen 106 William Smithfield 35.75 Programmer 12.6
15 Evergreen 102 David H. Senior 96.75 Systems Analyst 23.8
18 Amber Wave 114 Annelise Jones 48.10 Applications Designer 24.6
18 Amber Wave 118 James J. Frommer 18.36 General Support 45.3
18 Amber Wave 104 Anne K. Ramoras 96.75 Systems Analyst 32.4
18 Amber Wave 112 Darlene M. Smithson 45.95 DSS Analyst 44.0
22 Rolling Tide 105 Alice K. Johnson 105.00 Database Designer 64.7
22 Rolling Tide 104 Anne K. Ramoras 96.75 Systems Analyst 48.4
22 Rolling Tide 113 Delbert K. Joenbrood 48.10 Applications Designer 23.6
22 Rolling Tide 111 Geoff B. Wabash 26.87 Clerical Support 22.0
22 Rolling Tide 106 William Smithfield 35.75 Programmer 12.8
25 Starflight 107 Maria D. Alonzo 35.75 Programmer 24.6
25 Starflight 115 Travis B. Bawangi 96.75 Systems Analyst 45.8
25 Starflight 101 John G. News 105.00 Database Designer 56.3
25 Starflight 114 Annelise Jones 48.10 Applications Designer 33.1
25 Starflight 108 Ralph B. Washington 96.75 Systems Analyst 23.6
25 Starflight 118 James J. Frommer 18.36 General Support 30.5
25 Starflight 112 Darlene M. Smithson 45.95 DSS Analyst 41.4
25 Starflight 95 Darlene M. Smithson 96.75 Systems Analyst 23.6
"""

all_rows_new = []

for line in data_emp.split('\n')[1:]:
  # Use regular expression to match the pattern of each line
  match = re.match(
      r'(?P<PROJ_NUM>\d+)\s+'  # Project number: one or more digits
      r'(?P<PROJ_NAME>[\w\s]+?)\s+'  # Project name: word characters including spaces, non-greedy
      r'(?P<EMP_NUM>\d+)\s+'  # Employee number: one or more digits
      r'(?P<EMP_NAME>[\w\s.]+?)\s+'  # Employee name: word characters including spaces and dots, non-greedy
      r'(?P<CHG_HOUR>\d+\.\d+)\s+'  # Charge per hour: one or more digits, a dot, one or more digits
      r'(?P<JOB_CLASS>[\w\s.]+?)\s+'  # Job class: word characters including spaces and dots, non-greedy
      r'(?P<HOURS>\d+\.\d+)',  # Hours: one or more digits, a dot, one or more digits
      line.strip()
  )

  if match:
      row_data = match.groupdict()
      all_rows_new.append(row_data)

df = pd.DataFrame(all_rows_new)
df["CHG_HOUR"] = df["CHG_HOUR"].astype(float)
df["HOURS"] = df["HOURS"].astype(float)
df["EMP_NUM"] = df["EMP_NUM"].astype(int)
df["PROJ_NUM"] = df["PROJ_NUM"].astype(int)
df.head(22)

,PROJ_NUM,PROJ_NAME,EMP_NUM,EMP_NAME,CHG_HOUR,JOB_CLASS,HOURS
0,15,Evergreen,103,June E. Arbough,84.50,Elect. Engineer,23.8
1,15,Evergreen,101,John G. News,105.00,Database Designer,19.4
2,15,Evergreen,105,Alice K. Johnson,105.00,Database Designer,35.7
3,15,Evergreen,106,William Smithfield,35.75,Programmer,12.6
4,15,Evergreen,102,David H. Senior,96.75,Systems Analyst,23.8
5,18,Amber Wave,114,Annelise Jones,48.10,Applications Designer,24.6
6,18,Amber Wave,118,James J. Frommer,18.36,General Support,45.3
7,18,Amber Wave,104,Anne K. Ramoras,96.75,Systems Analyst,32.4
8,18,Amber Wave,112,Darlene M. Smithson,45.95,DSS Analyst,44.0
9,22,Rolling Tide,105,Alice K. Johnson,105.00,Database Designer,64.7


### 1) Utilisez ChatGPT pour créer une fonction qui donne les clés candidates de cette relation

In [5]:
from itertools import combinations

def find_candidate_keys(df):
    """
    Trouve les clés candidates d'un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame représentant la relation.

    Returns:
        list: Liste des clés candidates (chacune représentée par un ensemble d'attributs).
    """
    candidate_keys = []
    columns = df.columns.tolist()

    # Teste toutes les combinaisons possibles d'attributs
    for r in range(1, len(columns) + 1):
        for combo in combinations(columns, r):
            # Vérifie si la combinaison est une clé candidate
            if df[list(combo)].drop_duplicates().shape[0] == df.shape[0]:
                # Vérifie si elle est minimale (aucune sous-partie ne doit être une clé)
                is_minimal = all(
                    not set(subset).issubset(combo) or subset == combo
                    for subset in candidate_keys
                )
                if is_minimal:
                    candidate_keys.append(combo)
    return candidate_keys

# Utilisation
candidate_keys = find_candidate_keys(df)
print("Clés candidates :", candidate_keys)

Clés candidates : [('PROJ_NUM', 'EMP_NUM'), ('PROJ_NAME', 'EMP_NUM'), ('EMP_NUM', 'HOURS'), ('EMP_NAME', 'HOURS'), ('PROJ_NUM', 'EMP_NAME', 'CHG_HOUR'), ('PROJ_NUM', 'EMP_NAME', 'JOB_CLASS'), ('PROJ_NAME', 'EMP_NAME', 'CHG_HOUR'), ('PROJ_NAME', 'EMP_NAME', 'JOB_CLASS')]


### 2) Utilisez ChatGPT pour créer deux fonctions : une qui renvoie les attributs premiers, et l'autre les attributs non premiers

In [6]:
# Fonction 1
def get_prime_attributes(df):
    """
    Trouve les attributs premiers dans un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame représentant la relation.

    Returns:
        set: Ensemble des attributs premiers.
    """
    candidate_keys = find_candidate_keys(df)
    prime_attributes = set()

    for key in candidate_keys:
        prime_attributes.update(key)

    return prime_attributes

def get_non_prime_attributes(df):
    """
    Trouve les attributs non premiers dans un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame représentant la relation.

    Returns:
        set: Ensemble des attributs non premiers.
    """
    prime_attributes = get_prime_attributes(df)
    all_attributes = set(df.columns)
    non_prime_attributes = all_attributes - prime_attributes

    return non_prime_attributes

prime_attributes = get_prime_attributes(df)
non_prime_attributes = get_non_prime_attributes(df)

print("Attributs premiers :", prime_attributes)
print("Attributs non premiers :", non_prime_attributes)

Attributs premiers : {'EMP_NUM', 'CHG_HOUR', 'HOURS', 'JOB_CLASS', 'EMP_NAME', 'PROJ_NAME', 'PROJ_NUM'}
Attributs non premiers : set()


### 3) Utilisez ChatGPT pour créer une fonction qui vérifie s'il y a des dépendances partielles et les affiche

In [7]:
def find_partial_dependencies(df):
    """
    Trouve les dépendances partielles dans un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame représentant la relation.

    Returns:
        list: Liste des dépendances partielles sous la forme de tuples
              (sous-ensemble de la clé candidate, attribut dépendant).
    """
    candidate_keys = find_candidate_keys(df)
    partial_dependencies = []

    # Parcourt chaque clé candidate
    for key in candidate_keys:
        key_set = set(key)
        # Vérifie tous les sous-ensembles propres de la clé candidate
        for r in range(1, len(key)):
            for subset in combinations(key, r):
                subset_set = set(subset)
                # Attributs dépendants : tous les attributs non dans la clé candidate
                dependent_attributes = set(df.columns) - key_set
                for attr in dependent_attributes:
                    # Vérifie si l'attribut dépend fonctionnellement du sous-ensemble
                    if df[list(subset) + [attr]].drop_duplicates().shape[0] == df[list(subset)].drop_duplicates().shape[0]:
                        partial_dependencies.append((subset, attr))

    return partial_dependencies

partial_dependencies = find_partial_dependencies(df)
if partial_dependencies:
    print("Dépendances partielles trouvées :")
    for subset, attr in partial_dependencies:
        print(f"{subset} → {attr}")
else:
    print("Aucune dépendance partielle trouvée.")

Dépendances partielles trouvées :
('PROJ_NUM',) → PROJ_NAME
('EMP_NUM',) → CHG_HOUR
('EMP_NUM',) → JOB_CLASS
('EMP_NUM',) → EMP_NAME
('PROJ_NAME',) → PROJ_NUM
('EMP_NUM',) → CHG_HOUR
('EMP_NUM',) → JOB_CLASS
('EMP_NUM',) → EMP_NAME
('EMP_NUM',) → CHG_HOUR
('EMP_NUM',) → JOB_CLASS
('EMP_NUM',) → EMP_NAME
('PROJ_NUM',) → PROJ_NAME
('CHG_HOUR',) → JOB_CLASS
('PROJ_NUM', 'EMP_NAME') → PROJ_NAME
('PROJ_NUM', 'CHG_HOUR') → PROJ_NAME
('PROJ_NUM', 'CHG_HOUR') → JOB_CLASS
('EMP_NAME', 'CHG_HOUR') → EMP_NUM
('EMP_NAME', 'CHG_HOUR') → JOB_CLASS
('PROJ_NUM',) → PROJ_NAME
('JOB_CLASS',) → CHG_HOUR
('PROJ_NUM', 'EMP_NAME') → PROJ_NAME
('PROJ_NUM', 'JOB_CLASS') → PROJ_NAME
('PROJ_NUM', 'JOB_CLASS') → CHG_HOUR
('EMP_NAME', 'JOB_CLASS') → EMP_NUM
('EMP_NAME', 'JOB_CLASS') → CHG_HOUR
('PROJ_NAME',) → PROJ_NUM
('CHG_HOUR',) → JOB_CLASS
('PROJ_NAME', 'EMP_NAME') → PROJ_NUM
('PROJ_NAME', 'CHG_HOUR') → PROJ_NUM
('PROJ_NAME', 'CHG_HOUR') → JOB_CLASS
('EMP_NAME', 'CHG_HOUR') → EMP_NUM
('EMP_NAME', 'CHG_HOUR')

### 4) Utilisez ChatGPT pour créer une fonction qui vérifie s'il y a des dépendances transitives et les affiche

In [8]:
def find_transitive_dependencies(df):
    """
    Trouve les dépendances transitives dans un DataFrame.

    Args:
        df (pd.DataFrame): Le DataFrame représentant la relation.

    Returns:
        list: Liste des dépendances transitives sous la forme de tuples
              (attribut source, attribut intermédiaire, attribut cible).
    """
    candidate_keys = find_candidate_keys(df)
    prime_attributes = get_prime_attributes(df)
    transitive_dependencies = []

    # Vérifie les dépendances transitives parmi les colonnes
    all_attributes = set(df.columns)
    non_prime_attributes = all_attributes - prime_attributes

    # Parcourt les attributs non premiers
    for attr_c in non_prime_attributes:
        for attr_b in all_attributes:
            if attr_b == attr_c:
                continue

            # Vérifie si attr_b → attr_c
            if df[[attr_b, attr_c]].drop_duplicates().shape[0] == df[[attr_b]].drop_duplicates().shape[0]:
                for attr_a in all_attributes:
                    if attr_a == attr_b or attr_a == attr_c:
                        continue

                    # Vérifie si attr_a → attr_b mais pas attr_a → attr_c
                    if (
                        df[[attr_a, attr_b]].drop_duplicates().shape[0] == df[[attr_a]].drop_duplicates().shape[0]
                        and df[[attr_a, attr_c]].drop_duplicates().shape[0] != df[[attr_a]].drop_duplicates().shape[0]
                    ):
                        transitive_dependencies.append((attr_a, attr_b, attr_c))

    return transitive_dependencies

transitive_dependencies = find_transitive_dependencies(df)
if transitive_dependencies:
    print("Dépendances transitives trouvées :")
    for source, intermediary, target in transitive_dependencies:
        print(f"{source} → {intermediary} → {target}")
else:
    print("Aucune dépendance transitive trouvée.")

Aucune dépendance transitive trouvée.


### 5) a) Lister toutes les dépendances fonctionnelles dans cette relation
PROJ_NUM, EMP_NUM → PROJ_NAME  
EMP_NUM → EMP_NAME  
EMP_NUM → JOB_CLASS  
EMP_NUM → CHG_HOUR  
PROJ_NUM, EMP_NUM → HOURS  

### b) Expliquez pourquoi cette relation est sous la première forme normale.

il n'y a pas de valeurs composites ou de groupes répétitifs

### 6) a) Expliquez pourquoi cette relation n'est pas sous la deuxième forme normale.
Tout les attributs non premiers ne ne dépendent pas complètement de la clé primaire donc règle de la 2FN non respectée
### b) Convertissez cette relation en 2FN.  
***
Relation Employee :

EMP_NUM (clé primaire)
EMP_NAME
JOB_CLASS

***

Relation ProjectEmployee :

PROJ_NUM, EMP_NUM (clé primaire composée)
PROJ_NAME
CHG_HOUR

***

### 7) a) Expliquez pourquoi cette relation n'est pas dans la troisième forme normale.

Dans un premier temps car elle n'est déjà pas en 2NF, et dans un second temps car il exsite des dépendances transitives entre les attributs EMP_NAME, JOB_CLASS et EMP_NUM et ces attributs dépendent indirectement de la clé primaire ce qui ne va pas avec les principes de la 3FN
### b) Convertissez cette relation en 3FN.
---
Relation Employee :

Clé primaire : EMP_NUM
Attributs : EMP_NUM, EMP_NAME, JOB_CLASS  

---

Relation ProjectEmployee :

Clé primaire : PROJ_NUM, EMP_NUM
Attributs : PROJ_NUM, EMP_NUM, PROJ_NAME, CHG_HOUR, HOURS  
***


### 8) Fournissez le script SQL pour créer les tables sous la troisième forme normale en plus du script d'insertion.

#### Vous pouvez utiliser ChatGPT pour générer les scripts ci-dessus.

In [ ]:
# Création des tables :
CREATE TABLE Employee (
    EMP_NUM INT PRIMARY KEY,
    EMP_NAME VARCHAR(100),
    JOB_CLASS VARCHAR(100)
);

CREATE TABLE ProjectEmployee (
    PROJ_NUM INT,
    EMP_NUM INT,
    PROJ_NAME VARCHAR(100),
    CHG_HOUR FLOAT,
    HOURS FLOAT,
    PRIMARY KEY (PROJ_NUM, EMP_NUM),
    FOREIGN KEY (EMP_NUM) REFERENCES Employee(EMP_NUM)
);

# Insertion des données
INSERT INTO Employee (EMP_NUM, EMP_NAME, JOB_CLASS) VALUES
(101, 'John G. News', 'Database Designer'),
(102, 'David H. Senior', 'Systems Analyst'),
(103, 'June E. Arbough', 'Elect. Engineer'),
(104, 'Anne K. Ramoras', 'Systems Analyst'),
(105, 'Alice K. Johnson', 'Database Designer'),
(106, 'William Smithfield', 'Programmer'),
(107, 'Maria D. Alonzo', 'Programmer'),
(108, 'Ralph B. Washington', 'Systems Analyst'),
(111, 'Geoff B. Wabash', 'Clerical Support'),
(112, 'Darlene M. Smithson', 'DSS Analyst'),
(113, 'Delbert K. Joenbrood', 'Applications Designer'),
(114, 'Annelise Jones', 'Applications Designer'),
(115, 'Travis B. Bawangi', 'Systems Analyst'),
(118, 'James J. Frommer', 'General Support');

INSERT INTO ProjectEmployee (PROJ_NUM, EMP_NUM, PROJ_NAME, CHG_HOUR, HOURS) VALUES
(15, 103, 'Evergreen', 84.50, 23.8),
(15, 101, 'Evergreen', 105.00, 19.4),
(15, 105, 'Evergreen', 105.00, 35.7),
(15, 106, 'Evergreen', 35.75, 12.6),
(15, 102, 'Evergreen', 96.75, 23.8),
(18, 114, 'Amber Wave', 48.10, 24.6),
(18, 118, 'Amber Wave', 18.36, 45.3),
(18, 104, 'Amber Wave', 96.75, 32.4),
(18, 112, 'Amber Wave', 45.95, 44.0),
(22, 105, 'Rolling Tide', 105.00, 64.7),
(22, 104, 'Rolling Tide', 96.75, 48.4),
(22, 113, 'Rolling Tide', 48.10, 23.6),
(22, 111, 'Rolling Tide', 26.87, 22.0),
(22, 106, 'Rolling Tide', 35.75, 12.8),
(25, 107, 'Starflight', 35.75, 24.6),
(25, 115, 'Starflight', 96.75, 45.8),
(25, 101, 'Starflight', 105.00, 56.3),
(25, 114, 'Starflight', 48.10, 33.1),
(25, 108, 'Starflight', 96.75, 23.6),
(25, 118, 'Starflight', 18.36, 30.5),
(25, 112, 'Starflight', 45.95, 41.4),
(25, 95, 'Starflight', 96.75, 23.6);